# Upwelling visualization

***

In [149]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import animation
from cmocean import cm
import os
import gsw
from datetime import datetime, timedelta
from dateutil.parser import parse
from salishsea_tools import viz_tools
from itertools import repeat
from warnings import simplefilter
from tqdm import tqdm_notebook as tqdm
from dynmodes import dynmodes
from IPython.display import HTML

%matplotlib inline
plt.rcParams['font.size'] = 12
plt.rcParams['animation.html'] = 'html5'
simplefilter('ignore')

***

In [279]:
# Define path and filename string
path = '/data/bmoorema/results/Lake/S4d'
fn = 'SalishSeaIdeal_1h_20170701_20170706_grid_T.nc'
mask = xr.open_dataset(os.path.join(path, 'SalishSeaPond_S4d05ms_halocline10m_rhosurf1016/mesh_mask.nc')).sel(t=0)
subdomain = [114, 398, 334, 898]
const = {'g': 9.81, 'f': 1.11e-4, 'rho_0': 1024, 'H': 200}

# Define scaling parameters
param = {
    'u_wind': [5, 10, 15],
    'tau': [0.034, 0.123, 0.280],
    'z_h': [10, 15, 20],
    'rho_surf': [1016, 1018, 1020],
}

In [76]:
def plot_salinity(hour):
    
    salinity = np.ma.masked_where(mask.tmask[10, ...] == 0, data.vosaline[hour, 10, ...])
    c = ax.contourf(data.x, data.y, salinity, levels=np.arange(20, 31, 1), cmap=cm.haline, extend='both')
        
    return c

In [118]:
%%capture
fig, ax = plt.subplots(1, 1, figsize=(5, 10))
cax = fig.add_axes([0.92, 0.15, 0.03, 0.7])
ax.contourf(mask.x, mask.y, mask.tmask[0, ...], levels=[-0.1, 0.1], colors='Burlywood')
ax.contour(mask.x, mask.y, mask.tmask[0, ...], levels=[-0.1, 0.1], colors='k')
ax.set_xlim([0, 230])
ax.set_ylim([0, 410])
viz_tools.set_aspect(ax)

# Loop though z_h and rho_surf
z_h = param['z_h'][0]
rho_s = param['rho_surf'][0]
U = param['u_wind'][0]

# Open results record
runID = f'SalishSeaPond_S4d{U:02d}ms_halocline{z_h:2d}m_rhosurf{rho_s:4d}'
data = xr.open_dataset(os.path.join(path, runID, fn))
c = plot_salinity(0)    
fig.colorbar(c, cax=cax)

def animate(hour):
    
    plot_salinity(hour)
    ax.set_title(f'Hour {hour}')

anim = animation.FuncAnimation(fig, animate, frames=143, interval=100)

In [119]:
HTML(anim.to_html5_video())